# Lesson 22: Foundations of the Writing Workflow — Brown Agent

In this lesson, we'll dive deep into the Brown writing agent, a sophisticated system designed to generate high-quality technical articles about AI. This lesson focuses on the foundational components that make the writing workflow effective:

- **The orchestrator-worker pattern** for generating media assets like diagrams
- **Context engineering** techniques for providing rich context to the article writer
- **Entity modeling** using Pydantic to structure guidelines, research, profiles, and media
- **Custom node abstractions** for building workflow components
- **Markdown manipulation** utilities for handling content

Learning Objectives:
- Understand how to engineer context for high-quality content generation
- Learn the orchestrator-worker pattern for delegating specialized tasks
- Master entity modeling with mixins for flexible context representation
- Build custom workflow nodes with tool integration
- Work with Markdown content programmatically


## 1. Setup

First, we define some standard Magic Python commands to autoreload Python packages whenever they change:


In [20]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set Up Python Environment

To set up your Python virtual environment using `uv` and load it into the Notebook, follow the step-by-step instructions from the `Course Admin` lesson from the beginning of the course.

**TL/DR:** Be sure the correct kernel pointing to your `uv` virtual environment is selected.


### Configure Gemini API

To configure the Gemini API, follow the step-by-step instructions from the `Course Admin` lesson.

But here is a quick check on what you need to run this Notebook:

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/apikey).
2.  From the root of your project, run: `cp .env.example .env` 
3.  Within the `.env` file, fill in the `GOOGLE_API_KEY` variable:

Now, the code below will load the key from the `.env` file:


In [21]:
from utils import env

env.load(required_env_vars=["GOOGLE_API_KEY"])


Environment variables loaded from `/Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.env`
Environment variables loaded successfully.


### Import Key Packages


In [22]:
import nest_asyncio

nest_asyncio.apply()  # Allow nested async usage in notebooks


## 2. Download Required Resources

Before we start, we need to download the necessary configuration files, profiles, examples, and test data. These resources contain the content generation profiles, few-shot examples, and test inputs we'll use throughout this lesson.


### Download Configs


In [ ]:
!rm -rf configs
!curl -L -o configs.zip https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/data/configs.zip
!unzip configs.zip
!rm -rf configs.zip

I0000 00:00:1763830787.372942  916673 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

I0000 00:00:1763830787.624038  916673 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


100  1239  100  1239    0     0   7244      0 --:--:-- --:--:-- --:--:--  7288
Archive:  configs.zip
   creating: configs
  inflating: configs/debug.yaml      
  inflating: configs/course.yaml     


I0000 00:00:1763830788.049297  916673 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1763830788.299086  916673 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


### Download Inputs


In [24]:
!rm -rf inputs
!curl -L -o inputs.zip https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/data/inputs.zip
!unzip inputs.zip
!rm -rf inputs.zip


I0000 00:00:1763829889.765284  916673 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

I0000 00:00:1763829890.012438  916673 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


100 1410k  100 1410k    0     0  2918k      0 --:--:-- --:--:-- --:--:-- 2919k
Archive:  inputs.zip
   creating: inputs
   creating: inputs/evals
   creating: inputs/evals/dataset
  inflating: inputs/evals/dataset/metadata.json  
   creating: inputs/evals/dataset/data
   creating: inputs/evals/dataset/data/08_react_practice
  inflating: inputs/evals/dataset/data/08_react_practice/research.md  
  inflating: inputs/evals/dataset/data/08_react_practice/article_guideline.md  
  inflating: inputs/evals/dataset/data/08_react_practice/article_ground_truth.md  
   creating: inputs/evals/dataset/data/04_structured_outputs
  inflating: inputs/evals/dataset/data/04_structured_outputs/article_generated.md  
  inflating: inputs/evals/dataset/data/04_structured_outputs/research.md  
  inflating: inputs/evals/dataset/data/04_structured_outputs/article_guideline.md  
  inflating: inputs/evals/dataset/data/04_structured_outputs/article_ground_truth.md  
   creating: inputs/evals/dataset/data/05_workflo

I0000 00:00:1763829890.997430  916673 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1763829891.283910  916673 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


### Verify Downloaded Resources


In [25]:
%ls


article_guideline.md       inputs/
article_guideline_text.md  notebook.ipynb
configs/                   notebook_old.ipynb


I0000 00:00:1763829891.566757  916673 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


### Understanding the Brown Package

Throughout this notebook, we'll be importing code from the `brown` package. This package is the production version of the writing workflow located in `../writing_workflow/`. 

The Brown package is installed as a local Python package using `uv` and contains all the core functionality we'll explore:

- **Entities** (`brown.entities`): Pydantic models for articles, guidelines, research, profiles, and media
- **Nodes** (`brown.nodes`): Workflow components like ArticleWriter and MediaGeneratorOrchestrator
- **Models** (`brown.models`): LLM configuration and initialization utilities
- **Loaders** (`brown.loaders`): Classes to load markdown content into entities
- **Builders** (`brown.builders`): Factory patterns for creating loaders, renderers, and models
- **Renderers** (`brown.renderers`): Classes to save entities back to markdown files
- **Config** (`brown.config`): Settings management using Pydantic

Let's start by understanding how we load environment variables and configure the system.


### Configuration and Settings

The Brown agent uses a centralized settings system built with Pydantic's `BaseSettings`. This ensures type-safe configuration management and seamless integration with environment variables.

Let's examine the `Settings` class from `brown.config`:

```python
import os
from functools import lru_cache
from typing import Annotated

from loguru import logger
from pydantic import Field, FilePath, SecretStr
from pydantic_settings import BaseSettings, SettingsConfigDict

ENV_FILE_PATH = os.getenv("ENV_FILE_PATH", ".env")
logger.info(f"Loading environment file from `{ENV_FILE_PATH}`")


class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_file=ENV_FILE_PATH, extra="ignore", env_file_encoding="utf-8")

    # --- Gemini ---
    GOOGLE_API_KEY: SecretStr | None = Field(default=None, description="The API key for the Gemini API.")

    # --- Opik ---
    OPIK_WORKSPACE: str | None = Field(default=None, description="Name of the Opik workspace containing the project.")
    OPIK_PROJECT_NAME: str = Field(default="brown", description="Name of the Opik project.")
    OPIK_API_KEY: SecretStr | None = Field(default=None, description="The API key for the Opik API.")

    # --- App Config ---
    CONFIG_FILE: Annotated[FilePath, Field(default="configs/course.yaml", description="Path to the application configuration YAML file.")]


@lru_cache(maxsize=1)
def get_settings() -> Settings:
    return Settings()
```

**Key Design Patterns:**

1. **Singleton Pattern**: The `@lru_cache(maxsize=1)` decorator ensures we only instantiate `Settings` once, making it behave like a singleton throughout the application.

2. **Type Safety**: Using `SecretStr` for sensitive data like API keys ensures they're handled securely and not accidentally logged.

3. **Environment Integration**: `SettingsConfigDict` automatically loads values from the `.env` file, with sensible defaults for missing values.

4. **Flexible Configuration**: The `CONFIG_FILE` field points to a YAML configuration that controls workflow behavior, model selection, and more.

This pattern ensures consistent, type-safe access to configuration throughout the codebase.


## 3. How the Writing Agent Works: High-Level Overview

Before diving into the implementation details, let's understand the three-step workflow that powers the Brown writing agent. This systematic approach ensures high-quality article generation by properly managing context and delegating specialized tasks.

### The Three-Step Process

**Step 1: Load Context into Memory**

The first step involves gathering all the necessary context that will guide the article generation process:

- **Article Guideline**: The user's intent describing what the article should contain, its structure, and any specific requirements
- **Research**: Factual data, references, and information that will support the article's claims
- **Few-Shot Examples**: Sample articles demonstrating the desired writing style and structure
- **Content Generation Profiles**: Specialized profiles that control different aspects of the writing:
  - Character profile (voice and perspective)
  - Article profile (article-specific rules)
  - Structure profile (formatting rules)
  - Mechanics profile (writing mechanics)
  - Terminology profile (word choice)
  - Tonality profile (tone and style)

**Step 2: Generate Media Items (Orchestrator-Worker Pattern)**

Once we have the context, we need to generate any required media assets like diagrams or charts. This step uses the orchestrator-worker pattern:

- The **MediaGeneratorOrchestrator** analyzes the article guideline and research to identify what media items need to be generated
- For each identified requirement, the orchestrator delegates to specialized **worker tools** (e.g., `MermaidDiagramGenerator`)
- Workers generate their specialized content type in parallel
- All generated media items are collected and prepared for integration into the article

**Step 3: Write the Article**

With all context and media items ready, the **ArticleWriter** node generates the final article:

- Takes all the context from Step 1
- Integrates the media items from Step 2
- Follows all profile rules to generate content matching the desired style
- Produces a complete, high-quality article

Let's visualize this workflow with a diagram:
```mermaid
graph TD
    Step1["Step 1: Load Context"]
    
    Step1 --> LoadGuideline["Load Article Guideline"]
    Step1 --> LoadResearch["Load Research"]
    Step1 --> LoadExamples["Load Few-Shot Examples"]
    Step1 --> LoadProfiles["Load Profiles"]
    
    LoadGuideline --> Step2["Step 2: Media Generator Orchestrator Node"]
    LoadResearch --> Step2
    LoadExamples --> Step2
    LoadProfiles --> Step2
    
    Step2 --> DelegateWorkers["Delegate to Worker Tools"]
    
    DelegateWorkers --> Worker1
    DelegateWorkers --> Worker2
    DelegateWorkers --> Worker3
    
    Worker1 --> CollectMedia["Collect Generated Media"]
    Worker2 --> CollectMedia
    Worker3 --> CollectMedia
    
    CollectMedia --> Step3["Step 3: Article Writer Node"]
    
    Step3 --> GeneratedArticle["Generated Article"]
```

In [ ]:
from IPython.display import Image

Image(
    url="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/l22_3_steps_writing_workflow.png"
)

### Key Design Principles

This workflow embodies several important design principles:

1. **Separation of Concerns**: Each step has a clear, focused responsibility
2. **Context Engineering**: All relevant information is properly structured before generation
3. **Specialization**: Media generation is delegated to specialized workers
4. **Parallelization**: Media items are generated concurrently for efficiency
5. **Composability**: Each component can be tested and improved independently

Now that we understand the high-level workflow, let's dive into the implementation details of each component, starting with how we structure and load the context.


## 4. Context Components: Understanding Entities and Loaders

Now that we understand the high-level workflow, let's explore how we model and load the various context components. The Brown agent uses Pydantic models (entities) to represent different types of content, and specialized loaders to read this content from disk.

### Setting Up Directory Constants

First, let's define the directories where our context files are located:


In [41]:
from pathlib import Path

# Directory containing our test data
TEST_DIR = Path("inputs/tests/01_sample")

# Directory containing few-shot examples
EXAMPLES_DIR = Path("inputs/examples/course_lessons")

# Directory containing content generation profiles
PROFILES_DIR = Path("inputs/profiles")

### The ContextMixin: Foundation for Context Engineering

Before we dive into specific entities, let's understand the `ContextMixin` - a crucial abstraction that enables powerful context engineering throughout the system.

The `ContextMixin` provides a standardized way to convert any entity into a context representation surrounded by XML tags. This is essential for:

1. **Clear Boundaries**: XML tags clearly delineate different pieces of context in prompts
2. **Structured Prompts**: LLMs can easily parse and understand structured XML context
3. **Consistent Interface**: All entities follow the same pattern for context conversion
4. **Easy Composition**: Different context elements can be seamlessly combined

Here's the implementation from `brown.entities.mixins`:

```python
from abc import abstractmethod
from brown.utils.s import camel_to_snake


class ContextMixin:
    @property
    def xml_tag(self) -> str:
        return camel_to_snake(self.__class__.__name__)

    @abstractmethod
    def to_context(self) -> str:
        """Context representation of the object."""
        pass
```

**Key Features:**

- **Automatic Tag Generation**: The `xml_tag` property automatically converts the class name to snake_case (e.g., `ArticleGuideline` → `article_guideline`)
- **Abstract Method**: `to_context()` must be implemented by each entity, ensuring consistency
- **Flexible Implementation**: Each entity can customize how it represents itself as context

For example, an `ArticleGuideline` entity will be wrapped in `<article_guideline>...</article_guideline>` tags when passed to the LLM.


### Article Guideline Entity

The `ArticleGuideline` represents the user's intent - what they want the article to contain, how it should be structured, and any specific requirements. It's the primary driver of content generation.

From `brown.entities.guidelines`:

```python
from pydantic import BaseModel
from brown.entities.mixins import ContextMixin


class ArticleGuideline(BaseModel, ContextMixin):
    content: str

    def to_context(self) -> str:
        return f"""
<{self.xml_tag}>
    <content>{self.content}</content>
</{self.xml_tag}>
"""

    def __str__(self) -> str:
        return f"ArticleGuideline(len_content={len(self.content)})"
```

**Design Insights:**

1. **Simplicity**: Just a single `content` field containing the full guideline text
2. **ContextMixin Integration**: Implements `to_context()` to wrap content in XML tags
3. **Helpful String Representation**: Shows content length for debugging

The guideline typically contains:
- Article outline and section structure
- Specific instructions for each section
- Length constraints or requirements
- Important references to prioritize
- Any special formatting needs

Let's load an example article guideline:


In [44]:
from brown.loaders import MarkdownArticleGuidelineLoader
from utils import pretty_print

# Load the article guideline
guideline_loader = MarkdownArticleGuidelineLoader(uri=Path("article_guideline.md"))
article_guideline = guideline_loader.load(working_uri=TEST_DIR)

print(f"Loaded: {article_guideline}")
pretty_print.wrapped(article_guideline.content[:1500], title="Article Guideline (First 1500 Chars)", width=120)


Loaded: ArticleGuideline(len_content=23127)
----------------------------------------- Article Guideline (First 1500 Chars) -----------------------------------------
  ## Outline

1. Introduction: The Critical Decision Every AI Engineer Faces
2. Understanding the Spectrum: From Workflows to Agents
3. Choosing Your Path
4. Exploring Common Patterns
5. Zooming In on Our Favorite Examples
6. The Challenges of Every AI Engineer

## Section 1 - Introduction: The Critical Decision Every AI Engineer Faces

- **The Problem:** When building AI applications, engineers face a critical architectural decision early in their development process. Should they create a predictable, step-by-step workflow where they control every action, or should they build an autonomous agent that can think and decide for itself? This is one of the key decisions that will impact everything from the product such as development time and costs to reliability and user experience.
- **Why This Decision Matters:** Choose the 

### Research Entity

The `Research` entity contains factual data, references, and information that supports the article's claims. It can also extract and validate image URLs from the content.

From `brown.entities.research`:

```python
import re
from functools import cached_property

from loguru import logger
from pydantic import BaseModel

from brown.entities.mixins import ContextMixin
from brown.utils.a import asyncio_run, run_jobs
from brown.utils.network import is_image_url_valid


class Research(BaseModel, ContextMixin):
    content: str
    max_image_urls: int = 30

    @cached_property
    def image_urls(self) -> list[str]:
        # Extract image URLs using regex
        image_urls = re.findall(
            r"(?!data:image/)https?://[^\s]+\.(?:jpg|jpeg|png|bmp|webp)",
            self.content,
            re.IGNORECASE,
        )
        # Validate URLs asynchronously
        jobs = [is_image_url_valid(url) for url in image_urls]
        results = asyncio_run(run_jobs(jobs))

        urls = [url for url, valid in zip(image_urls, results) if valid]
        if len(urls) > self.max_image_urls:
            logger.warning(f"Found `{len(urls)} > {self.max_image_urls}` image URLs. Trimming to first {self.max_image_urls}.")
            urls = urls[: self.max_image_urls]

        return urls

    def to_context(self) -> str:
        return f"""
<{self.xml_tag}>
    {self.content}
</{self.xml_tag}>
"""

    def __str__(self) -> str:
        return f"Research(len_content={len(self.content)}, len_image_urls={len(self.image_urls)})"
```

**Advanced Features:**

1. **Image URL Extraction**: Automatically finds image URLs in the research content
2. **URL Validation**: Asynchronously validates that image URLs are accessible
3. **Caching**: Uses `@cached_property` to avoid re-extracting URLs
4. **Safety Limits**: Caps the number of images to prevent context overflow

The extracted image URLs can be passed to multimodal models (like Gemini) along with text prompts for richer context.

Let's load the research data:


In [45]:
from brown.loaders import MarkdownResearchLoader

research_loader = MarkdownResearchLoader(uri=Path("research.md"))
research = research_loader.load(working_uri=TEST_DIR)

print(f"Loaded: {research}")
pretty_print.wrapped(research.content[:1500], title="Research (First 1500 Chars)", width=120)


Loaded: Research(len_content=211792, len_image_urls=13)
--------------------------------------------- Research (First 1500 Chars) ---------------------------------------------
  # Research

## Code Sources

<details>
<summary>Repository analysis for https://github.com/google-gemini/gemini-cli/blob/main/README.md</summary>

# Repository analysis for https://github.com/google-gemini/gemini-cli/blob/main/README.md

## Summary
Repository: google-gemini/gemini-cli
File: README.md
Lines: 211

Estimated tokens: 1.6k

## File tree
```Directory structure:
└── README.md

```

## Extracted content
FILE: README.md
# Gemini CLI

 https://github.com/google-gemini/gemini-cli/actions/workflows/ci.yml/badge.svg ](https://github.com/google-gemini/gemini-cli/actions/workflows/ci.yml)

 ./docs/assets/gemini-screenshot.png 

This repository contains the Gemini CLI, a command-line AI workflow tool that connects to your
tools, understands your code and accelerates your workflows.

With the Gemini CLI you can

### Article Examples Entity

The `ArticleExamples` entity contains few-shot examples that demonstrate the desired writing style, structure, and quality. These serve as templates for the LLM to understand what kind of output is expected.

From `brown.entities.articles`:

```python
from pydantic import BaseModel
from brown.entities.mixins import ContextMixin


class ArticleExample(BaseModel, ContextMixin):
    content: str

    def to_context(self) -> str:
        return f"""
<{self.xml_tag}>
    {self.content}
</{self.xml_tag}>
"""

    def __str__(self) -> str:
        return f"ArticleExample(len_content={len(self.content)})"


class ArticleExamples(BaseModel, ContextMixin):
    examples: list[ArticleExample]

    def to_context(self) -> str:
        return f"""
<{self.xml_tag}>
        {"\n".join([example.to_context() for example in self.examples])}
</{self.xml_tag}>
"""
```

**Composition Pattern:**

- `ArticleExample`: Represents a single example article
- `ArticleExamples`: Contains multiple examples and composes their context representations
- When converted to context, all examples are nested within the parent XML tags

This pattern allows us to provide multiple examples to the LLM, showing consistency across different articles.

Let's load the few-shot examples:


In [ ]:
from brown.loaders import MarkdownArticleExampleLoader

examples_loader = MarkdownArticleExampleLoader(uri=EXAMPLES_DIR)
article_examples = examples_loader.load()

print(f"Loaded {len(article_examples.examples)} article examples")
if article_examples.examples:
    pretty_print.wrapped(
        article_examples.examples[0].content[:1500], title="First Article Example (First 1500 Chars)", width=120
    )


Loaded 2 article examples
--------------------------------------- First Article Example (First 1500 Chars) ---------------------------------------
  # Lesson 3: Context Engineering

AI applications have evolved rapidly. In 2022, we had simple chatbots for question-answering. By 2023, Retrieval-Augmented Generation (RAG) systems connected LLMs to domain-specific knowledge. 2024 brought us tool-using agents that could perform actions. Now, we are building memory-enabled agents that remember past interactions and build relationships over time.

In our last lesson, we explored how to choose between AI agents and LLM workflows when designing a system. As these applications grow more complex, prompt engineering, a practice that once served us well, is showing its limits. It optimizes single LLM calls but fails when managing systems with memory, actions, and long interaction histories. The sheer volume of information an agent might need, past conversations, user data, documents, and action de

### Profiles Entity

Profiles are the secret sauce of the Brown agent. They provide detailed instructions that control different aspects of content generation. The system uses multiple specialized profiles that work together.

From `brown.entities.profiles`:

```python
from pydantic import BaseModel
from brown.entities.mixins import ContextMixin


class Profile(BaseModel, ContextMixin):
    name: str
    content: str

    def to_context(self) -> str:
        return f"""
<{self.xml_tag}>
    <name>{self.name}</name>
    <content>{self.content}</content>
</{self.xml_tag}>
"""


class CharacterProfile(Profile):
    pass


class ArticleProfile(Profile):
    pass


class StructureProfile(Profile):
    pass


class MechanicsProfile(Profile):
    pass


class TerminologyProfile(Profile):
    pass


class TonalityProfile(Profile):
    pass


class ArticleProfiles(BaseModel):
    character: CharacterProfile
    article: ArticleProfile
    structure: StructureProfile
    mechanics: MechanicsProfile
    terminology: TerminologyProfile
    tonality: TonalityProfile
```

**Profile Hierarchy:**

1. **Base `Profile` Class**: Provides common structure (name + content) and context conversion
2. **Specialized Profile Classes**: Inherit from `Profile` to provide semantic meaning
3. **`ArticleProfiles` Container**: Holds all six profile types together

**Why Separate Profile Classes?**

Even though they have the same structure, separate classes provide:
- Type safety (can't accidentally swap profile types)
- Context clarity (each profile will have it's own XML tag when added as context)

Let's load all the profiles:


In [49]:
from brown.loaders import MarkdownArticleProfilesLoader

profiles_input = {
    "article": PROFILES_DIR / "article_profile.md",
    "character": PROFILES_DIR / "character_profiles" / "paul_iusztin.md",
    "mechanics": PROFILES_DIR / "mechanics_profile.md",
    "structure": PROFILES_DIR / "structure_profile.md",
    "terminology": PROFILES_DIR / "terminology_profile.md",
    "tonality": PROFILES_DIR / "tonality_profile.md",
}

profiles_loader = MarkdownArticleProfilesLoader(uri=profiles_input)
article_profiles = profiles_loader.load()

print("Profile Sizes (in characters):\n")
print(f"Character Profile: {len(article_profiles.character.content):,} characters")
print(f"Article Profile: {len(article_profiles.article.content):,} characters")
print(f"Structure Profile: {len(article_profiles.structure.content):,} characters")
print(f"Mechanics Profile: {len(article_profiles.mechanics.content):,} characters")
print(f"Terminology Profile: {len(article_profiles.terminology.content):,} characters")
print(f"Tonality Profile: {len(article_profiles.tonality.content):,} characters")
print(
    f"\nTotal Profile Content: {
        sum(
            [
                len(article_profiles.character.content),
                len(article_profiles.article.content),
                len(article_profiles.structure.content),
                len(article_profiles.mechanics.content),
                len(article_profiles.terminology.content),
                len(article_profiles.tonality.content),
            ]
        ):,} characters"
)

print()
pretty_print.wrapped(article_profiles.article.content[:1500], title="Article Profile (First 1500 Chars)", width=120)


Profile Sizes (in characters):

Character Profile: 3,033 characters
Article Profile: 13,074 characters
Structure Profile: 22,660 characters
Mechanics Profile: 4,747 characters
Terminology Profile: 10,730 characters
Tonality Profile: 4,192 characters

Total Profile Content: 58,436 characters

------------------------------------------ Article Profile (First 1500 Chars) ------------------------------------------
  ## Tonality

You should write in a humanized way as writing a blog article or book.

Write the description of ideas as fluid as possible. Remember that you are writing a book or blog article. Thus, everything should flow naturally, without too many bullet points or subheaders. Use them only when it really makes sense. Otherwise, stick to normal paragraphs.

## General Article Structure

The article is a collection of blocks that flow naturally one after the other. It starts with one introduction, continues with multiple sections in between and wrap-ups with a conclusion. The in

## 5. Deep Dive: Understanding Content Generation Profiles

Now that we've loaded all the profiles, let's understand what each one does and why it's essential for generating high-quality content. Profiles are the key to making the Brown agent produce consistent, high-quality output that matches your desired style and voice.

### Profile Categories

The profiles fall into two categories:

**1. General Profiles** (applicable to any content type):
- Mechanics Profile
- Structure Profile  
- Terminology Profile
- Tonality Profile

**2. Specialized Profiles** (specific to particular use cases):
- Article Profile (content-type specific)
- Character Profile (voice-specific)

Let's explore each profile and understand its role.


### 1. Mechanics Profile

**Purpose**: Controls the technical mechanics of writing words, sentences up to paragraphs.

**Location**: `inputs/profiles/mechanics_profile.md`

**What it covers**:
- Sentence length and complexity
- Paragraph structure and transitions
- Active vs passive voice usage
- Punctuation rules
- Readability guidelines

**Example rules**:
- "Vary sentence length for rhythm (mix short and long sentences)"
- "Use active voice unless passive is more appropriate"
- "Limit sentences to 20-25 words for clarity"

This profile ensures the writing is clean and easy to read.


### 2. Structure Profile

**Purpose**: Defines how the whole content should be organized and formatted. Instead of looking at the paragraph level, here we are interested at how the whole piece is structured.

**Location**: `inputs/profiles/structure_profile.md`

**What it covers**:
- Document structure (headings, sections)
- List formatting (when to use bullets vs numbers)
- Code block formatting
- Media placement rules
- Table of contents guidelines

**Example rules**:
- "Use ## for main sections, ### for subsections"
- "Code blocks must include language specification"
- "Use numbered lists for sequential steps, bullets for items"

This profile ensures consistent formatting across all generated content.


### 3. Terminology Profile

**Purpose**: Guides word choice and phrasing to match the target audience. Tries to avoid AI slop.

**Location**: `inputs/profiles/terminology_profile.md`

**What it covers**:
- Technical vs simple language decisions
- Jargon usage guidelines
- Acronym handling
- Preferred terminology
- Words to avoid

**Example rules**:
- "Define technical terms on first use"
- "Spell out acronyms on first mention: Machine Learning (ML)"
- "Avoid AI slop jargon like 'synergy' or 'leverage'"

This profile ensures the content speaks the right language for its audience.


### 4. Tonality Profile

**Purpose**: Sets the overall tone, voice, and style of the writing.

**Location**: `inputs/profiles/tonality_profile.md`

**What it covers**:
- Formality level
- Use of humor and personality
- Emotional tone
- Perspective (first/second/third person)
- Energy and enthusiasm level

**Example rules**:
- "Maintain a conversational but professional tone"
- "Use second person ('you') to engage readers"
- "Include occasional light humor when appropriate"

This profile gives the content its personality and makes it engaging.


### 5. Article Profile

**Purpose**: Defines rules specific to article generation (content-type specific).

**Location**: `inputs/profiles/article_profile.md`

**What it covers**:
- Article-specific structure requirements
- Introduction and conclusion patterns
- How to handle references and citations
- Examples and case study integration
- Call-to-action guidelines

**Example rules**:
- "Start with a hook that captures attention"
- "Include learning objectives near the beginning"
- "End with key takeaways and next steps"
- "Cite sources using footnotes or inline links"

This profile contains rules that only make sense for articles. For example, we could easily extend the agent to other formats such as video scripts or social media posts by adding `video_script_profile.md and `social_media_posts_profile.md`


### 6. Character Profile

**Purpose**: Injects a specific voice or persona into the writing.

**Location**: `inputs/profiles/character_profiles/paul_iusztin.md` (or other character files)

**What it covers**:
- Personal details (name, background)
- Professional expertise and experience
- Writing style preferences
- Content focus areas
- Unique phrases or expressions

**Example rules**:
- "Write from Paul's perspective as an AI engineer and educator"
- "Emphasize hands-on implementation over theory"
- "Use examples from real projects when possible"

**Why Character Profiles Matter:**

Character profiles allow the agent to write in a consistent voice:
- **Personal Brand**: Content feels like it comes from a real person
- **Authenticity**: References experiences and perspectives naturally
- **Flexibility**: Can switch between different voices (e.g., Paul Iusztin vs.Louis-Francois vs. Richard Feynman)
- **Consistency**: Maintains the same voice across multiple articles

You can create profiles for yourself or emulate famous figures' writing styles.


### How Profiles Work Together

The magic happens when all six profiles work together:

1. **Character Profile** → Establishes whose voice we're writing in
2. **Tonality Profile** → Sets the emotional tone and energy
3. **Terminology Profile** → Chooses the right words for the audience
4. **Mechanics Profile** → Ensures technical writing quality
5. **Structure Profile** → Organizes content logically
6. **Article Profile** → Applies article-specific best practices

**Example: How Profiles Shape a Single Paragraph**

Without profiles, an LLM might write:
> "Machine learning models require data. You need to preprocess this data. Then you train the model."

With all profiles applied:
> "Here's the thing about ML models - they're hungry for data, but not just any data. Before you can train your model, you'll need to clean and transform your raw data into a format the model can actually learn from. Think of it like preparing ingredients before cooking: the better your prep work, the better your final dish."

The profiles transform bland, generic text into engaging, personable, and technically accurate content that sounds like a real expert talking to a friend.


## 6. Model Configuration: Flexible LLM Management

Before we dive into the workflow nodes, let's understand how the Brown agent manages LLM configuration. The system provides a flexible way to configure different models for different nodes, allowing you to optimize cost and performance.

### The `get_model` Function

The central function for model initialization is `get_model` from `brown.models.get_model`. It provides a unified interface for creating LLM instances across the codebase.

From `brown.models.get_model`:

```python
import json

from langchain.chat_models import init_chat_model
from langchain_core.language_models import BaseChatModel

from brown.config import get_settings
from brown.models.config import DEFAULT_MODEL_CONFIGS, ModelConfig, SupportedModels
from brown.models.fake_model import FakeModel

MODEL_TO_REQUIRED_API_KEY = {
    SupportedModels.GOOGLE_GEMINI_30_PRO: "GOOGLE_API_KEY",
    SupportedModels.GOOGLE_GEMINI_25_PRO: "GOOGLE_API_KEY",
    SupportedModels.GOOGLE_GEMINI_25_FLASH: "GOOGLE_API_KEY",
    SupportedModels.GOOGLE_GEMINI_25_FLASH_LITE: "GOOGLE_API_KEY",
}


def get_model(model: SupportedModels, config: ModelConfig | None = None) -> BaseChatModel:
    if model == SupportedModels.FAKE_MODEL:
        if config and config.mocked_response is not None:
            if hasattr(config.mocked_response, "model_dump"):
                mocked_response_json = config.mocked_response.model_dump(mode="json")
            else:
                mocked_response_json = json.dumps(config.mocked_response)
            return FakeModel(responses=[mocked_response_json])
        else:
            return FakeModel(responses=[])

    config = config or DEFAULT_MODEL_CONFIGS.get(model) or ModelConfig()
    model_kwargs = {
        "model": model.value,
        **config.model_dump(),
    }

    required_api_key = MODEL_TO_REQUIRED_API_KEY.get(model)
    if required_api_key:
        settings = get_settings()
        if not getattr(settings, required_api_key):
            raise ValueError(f"Required environment variable `{required_api_key}` is not set")
        else:
            model_kwargs["api_key"] = getattr(settings, required_api_key)

    return init_chat_model(**model_kwargs)
```

**Key Features:**

1. **Fake Model Support**: For testing without API calls
2. **API Key Management**: Automatically pulls credentials from settings
3. **Default Configurations**: Falls back to sensible defaults
4. **LangChain Integration**: Uses `init_chat_model` for consistent interface


### Model Configuration Structures

The system uses three key structures for model configuration:

From `brown.models.config`:

```python
from enum import StrEnum
from typing import Any

from pydantic import BaseModel, Field


class SupportedModels(StrEnum):
    GOOGLE_GEMINI_30_PRO = "google_genai:gemini-3-pro-preview"
    GOOGLE_GEMINI_25_PRO = "google_genai:gemini-2.5-pro"
    GOOGLE_GEMINI_25_FLASH = "google_genai:gemini-2.5-flash"
    GOOGLE_GEMINI_25_FLASH_LITE = "google_genai:gemini-2.5-flash-lite"
    FAKE_MODEL = "fake"


class ModelConfig(BaseModel):
    temperature: float = 0.7
    top_k: int | None = None
    n: int = 1
    response_modalities: list[str] | None = None
    include_thoughts: bool = False
    thinking_budget: int | None = Field(
        default=None,
        ge=0,
        description="If reasoning is available, the maximum number of tokens the model can use for thinking.",
    )
    max_output_tokens: int | None = None
    max_retries: int = 6

    mocked_response: Any | None = None


DEFAULT_MODEL_CONFIGS = {
    "google_genai:gemini-2.5-pro": ModelConfig(
        temperature=0.7,
        include_thoughts=False,
        thinking_budget=1000,
        max_retries=3,
    ),
    "google_genai:gemini-2.5-flash": ModelConfig(
        temperature=1,
        thinking_budget=1000,
        include_thoughts=False,
        max_retries=3,
    ),
    "google_genai:gemini-2.0-flash-exp": ModelConfig(
        temperature=0.7,
        thinking_budget=1000,
        include_thoughts=False,
        max_retries=3,
    ),
}
```

**Design Insights:**

1. **`SupportedModels` Enum**: Type-safe model selection
2. **`ModelConfig` Pydantic Model**: Validates configuration parameters
3. **`DEFAULT_MODEL_CONFIGS` Dict**: Pre-configured settings for each model

This allows different nodes in the workflow to use different models with different configurations, optimizing for specific tasks (e.g., using faster models for media generation, more powerful models for article writing).


### Usage Throughout the Codebase

Every node in the workflow uses `get_model` to instantiate its LLM. This provides:

- **Consistency**: Same interface everywhere
- **Flexibility**: Easy to swap models for different nodes
- **Configuration**: Centralized model settings
- **Testing**: Can use FakeModel for tests

Let's try it out with a simple example:


In [32]:
from brown.models import get_model, ModelConfig, SupportedModels

# Create a model instance with custom configuration
model_config = ModelConfig(temperature=0.5, max_output_tokens=100)
model = get_model(SupportedModels.GOOGLE_GEMINI_25_FLASH, config=model_config)

# Test it with a simple prompt
response = await model.ainvoke([{"role": "user", "content": "Say hello in one sentence!"}])
print(f"Model response: {response.content}")


Model response: Hello there!


## 7. Media Generation: The Orchestrator-Worker Pattern

Now let's explore one of the most interesting architectural patterns in the Brown agent: the orchestrator-worker pattern for media generation. This pattern efficiently delegates specialized tasks to expert workers.

### Understanding Node Abstractions

First, let's understand the base abstractions that all workflow nodes inherit from.

From `brown.nodes.base`:

```python
from abc import ABC, abstractmethod
from typing import Any, Iterable, Literal, TypedDict

from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool


class ToolCall(TypedDict):
    name: str
    args: dict[str, Any]
    id: str
    type: Literal["tool_call"]


class Toolkit(ABC):
    """Base class for toolkits following LangChain's toolkit pattern."""

    def __init__(self, tools: list[BaseTool]) -> None:
        self._tools: list[BaseTool] = tools
        self._tools_mapping: dict[str, BaseTool] = {tool.name: tool for tool in self._tools}

    def get_tools(self) -> list[BaseTool]:
        """Get all registered media item generation tools."""
        return self._tools.copy()

    def get_tools_mapping(self) -> dict[str, BaseTool]:
        """Get a mapping of tool names to tool instances."""
        return self._tools_mapping

    def get_tool_by_name(self, name: str) -> BaseTool | None:
        """Get a specific tool by name."""
        return self._tools_mapping.get(name)


class Node(ABC):
    def __init__(self, model: Runnable, toolkit: Toolkit) -> None:
        self.toolkit = toolkit
        self.model = self._extend_model(model)

    def _extend_model(self, model: Runnable) -> Runnable:
        # Can be overridden to bind tools, structured output, etc.
        return model

    def build_user_input_content(self, inputs: Iterable[str], image_urls: list[str] | None = None) -> list[dict[str, Any]]:
        """Build multimodal input content with optional images."""
        messages: list[dict[str, Any]] = []
        if image_urls:
            for image_url in image_urls:
                messages.append({"type": "image_url", "image_url": {"url": image_url}})
        
        for input_ in inputs:
            messages.append({"type": "text", "text": input_})

        return messages

    @abstractmethod
    async def ainvoke(self) -> Any:
        pass
```

**Key Abstractions:**

1. **`ToolCall`**: TypedDict representing a tool call (name, args, id)
2. **`Toolkit`**: Manages a collection of tools that can be passed to a node
3. **`Node`**: Base class for all workflow nodes
   - Takes a model and toolkit
   - Can extend the model (bind tools, structured output)
   - Supports multimodal input (text + images)
   - Abstract `ainvoke` method for execution


### The MediaGeneratorOrchestrator Node

The orchestrator analyzes the article guideline and research to identify what media items need generation, then delegates to specialized worker tools.

Key implementation details from `brown.nodes.media_generator.MediaGeneratorOrchestrator`:

**1. Class Initialization:**
```python
class MediaGeneratorOrchestrator(Node):
    system_prompt_template = "..."

    def __init__(
        self,
        article_guideline: ArticleGuideline,
        research: Research,
        model: Runnable,
        toolkit: Toolkit,
    ) -> None:
        self.article_guideline = article_guideline
        self.research = research
        super().__init__(model, toolkit)
```

**2. Model Extension (Tool Binding):**
```python
def _extend_model(self, model: Runnable) -> Runnable:
    model = cast(BaseChatModel, super()._extend_model(model))
    model = model.bind_tools(self.toolkit.get_tools(), tool_choice="any")
    return model
```

The orchestrator binds all available worker tools to the model, allowing it to call multiple tools.

**3. Async Invocation:**
```python
async def ainvoke(self) -> list[ToolCall]:
    system_prompt = self.system_prompt_template.format(
        article_guideline=self.article_guideline.to_context(),
        research=self.research.to_context(),
    )
    user_input_content = self.build_user_input_content(
        inputs=[system_prompt], 
        image_urls=self.research.image_urls
    )
    inputs = [{"role": "user", "content": user_input_content}]
    response = await self.model.ainvoke(inputs)
    
    if isinstance(response, AIMessage) and response.tool_calls:
        jobs = cast(list[ToolCall], response.tool_calls)
    else:
        jobs = []
    
    return jobs
```

Returns a list of `ToolCall` objects describing what media items to generate.

**4. System Prompt (excerpt):**

The orchestrator uses a detailed system prompt that instructs it to:
- Analyze the article guideline for media requirements
- Look for explicit indicators like "Render a Mermaid diagram"
- Call appropriate worker tools with detailed descriptions
- Handle cases where no media is needed

Here is the full system prompt attached to the node:
```python
class MediaGeneratorOrchestrator(Node):
    system_prompt_template = """You are an Media Generation Orchestrator responsible for analyzing article 
guidelines and research to identify what media items need to be generated for the article.

Your task is to:
1. Carefully analyze the article guideline and research content provided
2. Identify ALL explicit requests for media items (diagrams, charts, visual illustrations, etc.)
3. For each identified media requirement, call the appropriate tool to generate the media item
4. Provide clear, detailed descriptions for each media item based on the guideline requirements and research context

## Analysis Guidelines

**Look for these explicit indicators in the article guideline:**
- Direct mentions: "Render a Mermaid diagram", "Draw a diagram", "Create a visual", "Illustrate with", etc.
- Visual requirements: "diagram visually explaining", "chart showing", "figure depicting", "visual representation"
- Process flows: descriptions of workflows, architectures, data flows, or system interactions
- Structural elements: hierarchies, relationships, comparisons, or step-by-step processes

**Key places to look:**
- Section requirements and descriptions  
- Specific instructions mentioning visual elements
- Complex concepts that would benefit from visual explanation
- Architecture or system descriptions
- Process flows or workflows

## Tool Usage Instructions

You will call multiple tools to generate the media items. You will use the tool that is most appropriate for the media item you are 
generating.

For each identified media requirement:

**When to use MermaidDiagramGenerator:**
- Explicit requests for Mermaid diagrams
- System architectures and workflows
- Process flows and data pipelines  
- Organizational structures or hierarchies
- Flowcharts for decision-making processes
- Sequence diagrams for interactions
- Entity-relationship diagrams
- Class diagrams for software structures
- State diagrams for system states
- Mind maps for concept relationships

**Description Requirements:**
When calling tools, provide detailed descriptions that include:
- The specific purpose and context from the article guideline
- Key components that should be included based on the research
- The type of diagram most appropriate (flowchart, sequence, architecture, etc.)
- Specific elements, relationships, or flows to highlight
- Any terminology or technical details from the research

## Example Analysis Process

1. **Scan the guideline** for phrases like:
   - "Render a Mermaid diagram of..."
   - "Draw a diagram showing..."
   - "Illustrate the architecture..."
   - "Visual representation of..."

2. **For each found requirement:**
   - Extract the specific context and purpose
   - Identify what should be visualized
   - Determine the most appropriate diagram type
   - Craft a detailed description incorporating research insights

3. **Call the appropriate tool** with the comprehensive description

## Input Context

Here is the article guideline:
{article_guideline}

Here is the research:
{research}

## Your Response

Analyze the provided article guideline and research, then call the appropriate tools for each 
identified media item requirement. Each tool call should include a detailed description that ensures 
the generated media item will be relevant, accurate, and valuable for the article's educational goals.

If no explicit media requirements are found in the guideline, respond with: 
"No explicit media item requirements found in the article guideline."
"""
```


### The ToolNode Abstraction

Worker tools inherit from `ToolNode`, a special type of `Node` that can be converted into a LangChain tool.

From `brown.nodes.base`:

```python
class ToolNode(Node):
    def __init__(self, model: Runnable) -> None:
        super().__init__(model, toolkit=Toolkit(tools=[]))

    def as_tool(self) -> StructuredTool:
        return StructuredTool.from_function(
            coroutine=self.ainvoke,
            name=f"{camel_to_snake(self.__class__.__name__)}_tool",
        )

    @abstractmethod
    async def ainvoke(self) -> Any:
        pass
```

**Key Features:**

- Inherits from `Node` but has no tools itself (empty toolkit)
- `as_tool()` method converts the node into a LangChain `StructuredTool`
- The tool name is automatically derived from the class name
- The tool's coroutine is the node's `ainvoke` method

This allows any `ToolNode` to be easily integrated as a worker tool in an orchestrator's toolkit.

Still, this pattern can easily be extended with the posability of making each node a tool for another node, providing full composability between nodes.

### The MermaidDiagramGenerator Worker

Before running the `MediaGeneratorOrchestrator`, let's examine a concrete worker implementation from `brown.nodes.tool_nodes.MermaidDiagramGenerator`.

**1. Structured Output Model:**
```python
class GeneratedMermaidDiagram(BaseModel):
    content: str = Field(description="The Mermaid diagram code formatted in Markdown format as: ```mermaid\n[diagram content here]\n```")
    caption: str = Field(description="The caption, as a short description of the diagram.")
```

**2. Class Structure:**
```python
class MermaidDiagramGenerator(ToolNode):
    prompt_template = "..."

    def _extend_model(self, model: Runnable) -> Runnable:
        model = cast(BaseChatModel, super()._extend_model(model))
        model = model.with_structured_output(GeneratedMermaidDiagram, include_raw=False)
        return model

    async def ainvoke(self, description_of_the_diagram: str, section_title: str) -> MermaidDiagram:
        """Specialized tool to generate a mermaid diagram from a text description. This tool uses a specialized LLM to
        convert a natural language description into a mermaid diagram.

        Use this tool when you need to generate a mermaid diagram to explain a concept. Don't confuse mermaid diagrams,
        or diagrams in general, with media data, such as images, videos, audio, etc. Diagrams are rendered dynamically
        customized for each article, while media data are static data added as URLs from external sources.
        This tool is used explicitly to dynamically generate diagrams, not to add media data.

        Args:
            description_of_the_diagram: Natural language description of the diagram to generate.
            section_title: Title of the section that the diagram is for.

        Returns:
            The generated mermaid diagram code in Markdown format.

        Raises:
            Exception: If diagram generation fails.

        Examples:
        >>> description = "A flowchart showing data flowing from user input to database"
        >>> diagram = await generate_mermaid_diagram(description)
        >>> print(diagram)
        ```mermaid
        graph LR
            A[User Input] --> B[Processing]
            B --> C[(Database)]
        ```
        """

        try:
            response = await self.model.ainvoke(
                [
                    {
                        "role": "user",
                        "content": self.prompt_template.format(
                            description_of_the_diagram=description_of_the_diagram,
                        ),
                    }
                ]
            )

        except Exception as e:
            logger.exception(f"Failed to generate Mermaid diagram: {e}")

            return MermaidDiagram(
                location=section_title,
                content=f'```mermaid\ngraph TD\n    A["Error: Failed to generate diagram"]\n    A --> B["{str(e)}"]\n```',
                caption=f"Error: Failed to generate diagram: {str(e)}",
            )

        if not isinstance(response, GeneratedMermaidDiagram):
            raise InvalidOutputTypeException(GeneratedMermaidDiagram, type(response))

        return MermaidDiagram(
            location=section_title,
            content=response.content,
            caption=response.caption,
        )
```

**Key Features:**

- Uses structured output to ensure valid diagram generation
- The ainvoke() method has a carefully designed pydoc and signature that will be used when the node it's transformed into a tool used by the orchestrator.
- In case of error, we return a placeholder diagram to avoid failing the whole workflow because of a tool failure. Remember that we can run dozens of tools in parallel, thus the change of failure due to things such as rate limits is high.

The prompt template contains detailed instructions for generating valid Mermaid diagrams, including:
- Syntax rules (always use double quotes, never use semicolons)
- Examples of different diagram types (flowcharts, process flows)
- Common mistakes to avoid

Here is the full prompt template:
```python
class MermaidDiagramGenerator(ToolNode):
    prompt_template = """
You are a specialized agent that creates clean, readable Mermaid diagrams from text descriptions.

## Task
Generate a valid Mermaid diagram based on this description:
<description_of_the_diagram>
{description_of_the_diagram}
</description_of_the_diagram>

## Output Format
Return ONLY the Mermaid code block in this exact format:
``mermaid
[diagram content here]
``

## Diagram Types & Examples

### Process Flow
``mermaid
graph LR
    A["Input"] --> B["Process"] --> C["Output"]
    B --> D["Validation"]
    D -->|"Valid"| C
    D -->|"Invalid"| A
``mermaid


### Flowcharts (Most Common)
``mermaid
graph TD
    A["Start"] --> B{{"Decision"}}
    B -->|"Yes"| C["Action 1"]
    B -->|"No"| D["Action 2"]
    C --> E["End"]
    D --> E
``mermaid

...

## Syntax Rules
1. **Node Labels**: Use square brackets `[Label]` for rectangular nodes
2. **Decisions**: Use curly braces `{{Decision}}` for diamond shapes  
3. **Databases**: Use `[(Label)]` for cylindrical database shapes
4. **Circles**: Use `((Label))` for circular nodes
5. **Arrows**: Use `-->` for solid arrows, `-.->` for dotted arrows
6. **Labels on Arrows**: Use `-->|Label|` for labeled connections
7. **Subgraphs**: Use `subgraph "Title"` and `end` to group elements
8. **Comments**: Use `%%` for comments
9. **ERD Entities**: Use `ENTITY_NAME {{ field_type field_name }}` format
10. **ERD Relationships**: Use `||--o{{`, `||--||`, `}}o--||` for different cardinalities
11. **Class Definitions**: Use `class ClassName {{ +type attribute +method() }}` format
12. **Class Relationships**: Use `<|--` (inheritance), `-->` (association), `--*` (composition)

## Formatting Rules
**ALWAYS wrap node labels in double quotes `"..."` to prevent parsing errors:**

**WRONG** (causes parse errors):
h

**Key formatting requirements:**
- **Always use double quotes** around ALL node labels: `A["Label"]`
- **Never use semicolons** at the end of lines (they're optional and can cause issues)
- **Quote any label** containing: parentheses `()`, commas `,`, periods `.`, colons `:`, or spaces
- **Quote subgraph titles** as well: `subgraph "Title"`

## Styling Guidelines
- **Use default colors only** - do not add color specifications or custom styling
- Do not use `fill:`, `stroke:`, `color:` or any CSS styling properties
- Keep diagrams clean and professional with standard Mermaid appearance
- Focus on structure and clarity, not visual customization

## Key Guidelines
- Keep node labels concise (avoid parentheses and special characters in labels)
- Use clear, logical flow from top to bottom or left to right
- Keep the diagram simple and easy to understand
- Group related elements with subgraphs when helpful
- Maintain consistent spacing and formatting
- Choose the appropriate diagram type for the concept being illustrated

## Common Mistakes to Avoid
- **NEVER use unquoted labels** - always wrap in double quotes: `A["Label"]`
- **NEVER use semicolons** at the end of lines (causes parsing issues)
- **NEVER put parentheses `()` in unquoted labels** - parser treats them as shape tokens
- Don't create overly complex diagrams with too many connections
- Avoid extremely long labels that break the visual flow
- **Never use custom colors or styling** - stick to Mermaid's default appearance

Generate a clean, professional diagram that clearly illustrates the described concept using only default Mermaid 
styling. Remember: ALWAYS use double quotes around ALL labels and NEVER use semicolons.
"""
```


### Example: Generating Media Items with the Orchestrator

Let's see the orchestrator-worker pattern in action by generating Mermaid diagrams:


In [50]:
import asyncio

from brown.nodes import MediaGeneratorOrchestrator, MermaidDiagramGenerator, Toolkit
from brown.models import get_model, SupportedModels

# Create worker tool
diagram_model = get_model(SupportedModels.GOOGLE_GEMINI_25_FLASH)
mermaid_generator = MermaidDiagramGenerator(model=diagram_model)
toolkit = Toolkit(tools=[mermaid_generator.as_tool()])

# Create orchestrator
orchestrator_model = get_model(SupportedModels.GOOGLE_GEMINI_25_FLASH)
orchestrator = MediaGeneratorOrchestrator(
    article_guideline=article_guideline,
    research=research,
    model=orchestrator_model,
    toolkit=toolkit,
)

# Get media generation jobs
print("Analyzing article guideline for media requirements...\n")
media_jobs = await orchestrator.ainvoke()

print(f"Found {len(media_jobs)} media items to generate:\n")
for i, job in enumerate(media_jobs):
    print(f"{i + 1}. Tool: {job['name']}")
    print(f"   Description: {job['args'].get('description_of_the_diagram', 'N/A')[:100]}...")
    print(f"   Section: {job['args'].get('section_title', 'N/A')}\n")

# Execute jobs in parallel
print("Generating media items in parallel...")
coroutines = []
for job in media_jobs:
    tool = orchestrator.toolkit.get_tool_by_name(job["name"])
    if tool:
        coroutines.append(tool.ainvoke(job["args"]))

media_items = await asyncio.gather(*coroutines)

print(f"\nGenerated {len(media_items)} media items successfully!")


Analyzing article guideline for media requirements...

Found 11 media items to generate:

1. Tool: mermaid_diagram_generator_tool
   Description: A simple LLM Workflow showing a predefined sequence of steps: starting the process, an LLM call for ...
   Section: Understanding the Spectrum: From Workflows to Agents

2. Tool: mermaid_diagram_generator_tool
   Description: A diagram illustrating a simple Agentic System, highlighting its core components and their interacti...
   Section: Understanding the Spectrum: From Workflows to Agents

3. Tool: mermaid_diagram_generator_tool
   Description: A graph illustrating the trade-off between application reliability and the agent's level of control....
   Section: Choosing Your Path

4. Tool: mermaid_diagram_generator_tool
   Description: A flowchart illustrating the AI generation and human verification loop. The process starts with 'AI ...
   Section: Choosing Your Path

5. Tool: mermaid_diagram_generator_tool
   Description: A Mermaid diagram 

In [51]:
# Show first diagram
if media_items:
    print(f"\nFirst generated diagram:")
    print(f"Location: {media_items[0].location}")
    print(f"Caption: {media_items[0].caption}")
    print(f"\nDiagram code:\n{media_items[0].content}")


First generated diagram:
Location: Understanding the Spectrum: From Workflows to Agents
Caption: An LLM Workflow showing a sequence of steps from start to end, including an LLM call for tool invocation and tool execution.

Diagram code:
```mermaid
graph TD
    A["Start Process"] --> B["LLM Call for Tool Invocation"]
    B --> C["Execute Tools"]
    C --> D["End Process"]
```


## 8. The ArticleWriter Node: Bringing It All Together

The `ArticleWriter` is the heart of the content generation system. It takes all the context we've prepared and generates a high-quality article following all the profile rules.

### Class Structure

From `brown.nodes.article_writer.ArticleWriter`:

**1. Initialization:**
```python
class ArticleWriter(Node):
    system_prompt_template = """..."""
    
    def __init__(
        self,
        article_guideline: ArticleGuideline,
        research: Research,
        article_profiles: ArticleProfiles,
        media_items: MediaItems,
        article_examples: ArticleExamples,
        model: Runnable,
    ) -> None:
        super().__init__(model, toolkit=Toolkit(tools=[]))
        
        self.article_guideline = article_guideline
        self.research = research
        self.article_profiles = article_profiles
        self.media_items = media_items
        self.article_examples = article_examples
```


### The ainvoke Method

**2. Article Generation Logic:**
```python
async def ainvoke(self) -> Article | SelectedText:
    # Format the system prompt with all context
    system_prompt = self.system_prompt_template.format(
        article_guideline=self.article_guideline.to_context(),
        research=self.research.to_context(),
        article_profile=self.article_profiles.article.to_context(),
        character_profile=self.article_profiles.character.to_context(),
        mechanics_profile=self.article_profiles.mechanics.to_context(),
        structure_profile=self.article_profiles.structure.to_context(),
        terminology_profile=self.article_profiles.terminology.to_context(),
        tonality_profile=self.article_profiles.tonality.to_context(),
        media_items=self.media_items.to_context(),
        article_examples=self.article_examples.to_context(),
    )
    
    # Build multimodal input (text + images from research)
    user_input_content = self.build_user_input_content(
        inputs=[system_prompt], 
        image_urls=self.research.image_urls
    )
    
    inputs = [{"role": "user", "content": user_input_content}]
    
    # Generate the article
    written_output = await self.model.ainvoke(inputs)
    if not isinstance(written_output, AIMessage):
        raise InvalidOutputTypeException(AIMessage, type(written_output))
    written_output = cast(str, written_output.text)
    
    return Article(content=written_output)
```

The method:
1. Combines all context using `to_context()` methods
2. Supports multimodal input with research images]
3. Check if we get the expected output. As the article generation is a critical step within our workflow, we want to fail the whole workflow if this node fails.
4. Returns an `Article` entity


### The System Prompt (Key Sections)

The `ArticleWriter` uses an extensive system prompt that includes all the context. Here are the key sections (simplified for clarity):

**3. System Prompt Structure:**

```
You are Brown, a professional human writer specialized in writing technical, educative and informational articles
about AI.

Your task is to write a high-quality article, while providing you the following context:
- article guideline: the user intent
- research: the factual data
- article profile: rules specific to writing articles
- character profile: the character you will impersonate
- structure profile: Structure rules
- mechanics profile: Mechanics rules
- terminology profile: Terminology rules
- tonality profile: Tonality rules

## Character Profile
{character_profile}

## Research
{research}

## Article Examples
{article_examples}

## Tonality Profile
{tonality_profile}

## Terminology Profile
{terminology_profile}

## Mechanics Profile
{mechanics_profile}

## Structure Profile
{structure_profile}

## Media Items
{media_items}

## Article Profile
{article_profile}

## Article Guideline
{article_guideline}

## Chain of Thought
1. Plan the article outline
2. Write the article following the article outline and all the other constraints
3. Check if all the constraints are respected. Edit the article if not
4. Return ONLY the final version of the article
```

**Key Prompt Engineering Techniques:**

1. **Clear Role Definition**: "You are Brown, a professional human writer..."
2. **Structured Context**: Each piece of context has its own section that is automatically enclosed by XML tags due to the `ContextMixin.to_context()` interface. 
3. **Priority Guidance**: The "Article guideline" which is the user input, always has priority over everything else.
4. **Chain of Thought**: Explicit reasoning steps
5. **Constraint Adherence**: Multiple reminders to follow all profiles, to be 100% sure the LLM picks up on our core instructions.
6. **Grounding in Facts**: "Anchor exclusively in research, avoid internal knowledge" to avoid hallucinations

This comprehensive prompt ensures the LLM has all the information and instructions it needs to generate high-quality, consistent content.


## 9. Complete Workflow Example

Now let's put everything together and run the complete workflow to generate an article using our test data.


In [53]:
from brown.nodes import ArticleWriter
from brown.entities.media_items import MediaItems
from brown.renderers import MarkdownArticleRenderer

pretty_print.wrapped(" COMPLETE WORKFLOW: GENERATING AN ARTICLE", width=100)

# Step 1: We already loaded the context (guideline, research, profiles, examples)
print("\n✓ Step 1: Context loaded")
print(f"  - Article Guideline: {len(article_guideline.content)} characters")
print(f"  - Research: {len(research.content)} characters")
print(f"  - Profiles: 6 profiles loaded")
print(f"  - Examples: {len(article_examples.examples)} examples")

# Step 2: Generate media items (we already did this in the previous section)
# Let's create a MediaItems container from our generated items
print("\n✓ Step 2: Media items generated")
if "media_items" in locals() and media_items:
    media_items_entity = MediaItems.build(media_items)
    print(f"  - Generated {len(media_items)} Mermaid diagrams")
else:
    # For demo purposes, we'll use empty media items
    media_items_entity = MediaItems.build([])
    print(f"  - No media items generated (using empty MediaItems)")

# Step 3: Write the article
print("\n⏳ Step 3: Writing article (this may take 1-2 minutes)...")

writer_model = get_model(SupportedModels.GOOGLE_GEMINI_25_FLASH)
article_writer = ArticleWriter(
    article_guideline=article_guideline,
    research=research,
    article_profiles=article_profiles,
    media_items=media_items_entity,
    article_examples=article_examples,
    model=writer_model,
)

generated_article = await article_writer.ainvoke()

print(f"✓ Step 3: Article generated ({len(generated_article.content)} characters)")
pretty_print.wrapped(generated_article.content[:1500], title="Generated Article", width=120)
pretty_print.wrapped(" WORKFLOW COMPLETE!", width=100)


----------------------------------------------------------------------------------------------------
   COMPLETE WORKFLOW: GENERATING AN ARTICLE
----------------------------------------------------------------------------------------------------

✓ Step 1: Context loaded
  - Article Guideline: 23127 characters
  - Research: 211792 characters
  - Profiles: 6 profiles loaded
  - Examples: 2 examples

✓ Step 2: Media items generated
  - Generated 11 Mermaid diagrams

⏳ Step 3: Writing article (this may take 1-2 minutes)...
✓ Step 3: Article generated (31900 characters)
-------------------------------------------------- Generated Article --------------------------------------------------
  # Workflows vs. Agents: The AI Engineer's Critical Decision
### Navigating the Architectural Choices for Production-Ready AI

When building AI applications, engineers face a critical architectural decision early in development. Should they create a predictable, step-by-step workflow where every action is

Save the article:

In [56]:
output_path = TEST_DIR / "article.md"
renderer = MarkdownArticleRenderer()
renderer.render(generated_article, output_uri=output_path)

pretty_print.wrapped(f"Article saved to {output_path}", width=100)

----------------------------------------------------------------------------------------------------
  Article saved to inputs/tests/01_sample/article.md
----------------------------------------------------------------------------------------------------


Now check out the article and let us know what you think!

## 10. Future Steps and Key Takeaways

Congratulations! You've just learned the foundations of the Brown writing agent. Let's recap what we covered and look ahead to what's next.

### What We've Learned

**1. Context Engineering:**
- How to structure entities with Pydantic for type safety
- The `ContextMixin` pattern for XML-based context representation
- Loading markdown content into structured entities
- Composing multiple context elements seamlessly

**2. The Orchestrator-Worker Pattern:**
- `Node` and `ToolNode` base abstractions
- `Toolkit` for managing collections of tools
- `MediaGeneratorOrchestrator` analyzing requirements and delegating tasks
- `MermaidDiagramGenerator` as a specialized worker
- Parallel execution of multiple worker tasks

**3. The ArticleWriter Node:**
- Comprehensive system prompt engineering
- Integration of all context components
- Multimodal input support (text + images)
- Profile-driven content generation

**4. Supporting Infrastructure:**
- Flexible model configuration with `ModelConfig`
- Settings management with Pydantic
- Loaders and renderers for I/O operations
- Factory patterns for building components

### Key Design Principles

Throughout this lesson, we've seen several important design principles:

1. **Separation of Concerns**: Each component has a single, well-defined responsibility
2. **Composition Over Inheritance**: Entities compose context, nodes compose functionality
3. **Type Safety**: Pydantic models ensure data integrity
4. **Abstraction**: Base classes provide consistent interfaces
5. **Flexibility**: Easy to swap models, profiles, or workers


### Ideas for Extension

Now that you understand the foundations, here are some ways you could extend this system:

**1. Additional Media Workers:**
- `ImageGenerator`: Generate images using Google's image generation features
- `ChartGenerator`: Create data visualizations with matplotlib or plotly

**2. Content Type Variations:**
- Social media post generator (using different profiles)
- Email newsletter generator
- Technical documentation generator
- Video transcript generator

**3. Reduce Costs and Latency:**
- Cache constant inputs between LLM calls such as the research and profiles to avoid recomputing them
- Compress the research relative to the article guideline

**4. Advanced Profiles:**
- Industry-specific profiles (finance, healthcare, education)
- Add your own character profile instead of ours based on Paul Iusztin

### What's Next in the Course

In the upcoming lesson, we'll explore:

**Lesson 23: The Evaluator-Optimizer Pattern**
- How to automatically review and edit generated articles
- Implementing the `ArticleReviewer` node
- Glue everything together into a LangGraph workflow

### Resources

- **Brown Package Documentation**: Explore `../writing_workflow/README.md`
- **Profile Examples**: Check `inputs/profiles/` for more profile templates
- **Test Data**: Use `inputs/tests/` for additional examples
- **Configuration**: Review `configs/` for workflow configuration options
